# Import all the necessary **libraries**
* **OPENCV** - used to manage **video capture and optical flow**
* **MTCNN** - used to detect **faces**
* **DEEPFACE** - used to detect **facial expressions**
* **NUMPY** - used to manage the **visualization** of the output (in this project)

In [7]:
import cv2 # OpenCV - to manage video capture and optical flow
from mtcnn import MTCNN # MTCNN - to detect faces
from deepface import DeepFace # DeepFace - to detect facial expressions
import numpy as np # Numpy - here used to mangae the visualization of the output
import imutils # Imutils - to resize the frames

# Define a function to draw **dots and arrows** (for the optical flow)
* **drawDotsandArrows** - draw dots on each block and arrows where there is a significant movement (used for optical flow)
    + `frame_displayed`: frame on which arrows and dots will be drawn
    + `flow`: optical flow of the frame
    + `blockSize`: size of the blocks (on which dots will be drawn)

In [8]:
def drawDotsAndArrows(frame_displayed, flow, blockSize): # Draw dots on each block and arrows where there is movement
    h, w = frame_displayed.shape[:2]
    step = blockSize
    scale = 5  # Length of the arrow
    threshold = 1  # Filters minor movements
    frame_displayed = cv2.cvtColor(frame_displayed, cv2.COLOR_GRAY2BGR)

    for y in range(0, h, step):
        for x in range(0, w, step):
            fx, fy = flow[y, x] # Movement vector for each block (x,y)
            # Draw a white dot on each block
            cv2.circle(frame_displayed, (x, y), 1, (255, 0, 0), -1)
            # Draw a green arrow if the movement is significant
            if abs(fx) > threshold or abs(fy) > threshold:
                end_x = int(x + fx * scale) 
                end_y = int(y + fy * scale) 
                cv2.arrowedLine(frame_displayed, (x, y), (end_x, end_y), (0, 255, 0), 1, tipLength=0.5)

    return frame_displayed

# Facial and emotion **recognition** and computation of the **optical flow**

1. **Initialization** of
    1. **Variables for optical flow**
    2. **Webcam**
    3. MTCNN **detector**
    4. Variables for **emotion/facial recognition**
    5. Dictionary for emotion's **colors**
2. Computation of **optical flow**
3. Recognition of **faces and emotions**
4. **Visualization** of emotions detected and optical flow

In [9]:
# Initialize variables for optical flow
displayHeight = 480
blockSize = 16


# Initialize the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")

# Read the first frame
ret, prev_frame = cap.read()
if not ret:
    print("Error: No frame acquired.")
    cap.release()


# Initialize the MTCNN detector
detector = MTCNN()

# Initialize variables to manage the emotion analysis
frame_count = 0
frame_interval = 10 # Do the analysis every 10 frames

# Variables to store the data of the faces detected
face_data = []

# Dictionary for the colors associated with each emotion
emotion_colors = {
    'angry': (0, 0, 255),       # Red
    'disgust': (0, 255, 0),     # Green
    'fear': (137, 49, 66),      # Purple
    'happy': (72, 205, 255),    # Yellow
    'sad': (255, 127, 0),       # Light Blue
    'surprise': (255, 0, 255),  # Yellow
    'neutral': (128, 128, 128)  # Gray
}

# Resize the first frame captured previously
prev_frame = cv2.flip(imutils.resize(prev_frame, displayHeight),1)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)


while True:
    # Acquire a new frame
    ret, frame = cap.read()

    # Check if the frame was acquired correctly, otherwise exit the loop and return an error message
    if not ret:
        print("Errore nell'acquisizione del frame.")
        break


    #Computation of the optical flow

    # Resize the frame and convert it into a grayscale
    frame = cv2.flip(imutils.resize(frame, displayHeight),1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


    # Compute the optical flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 
                                        pyr_scale=0.5, levels=3, 
                                        winsize=25, iterations=3, 
                                        poly_n=5, poly_sigma=1.2, flags=0)

    # Update the previous frame 
    prev_gray = gray.copy()
    
    # Draw dots and arrows on the frame
    flow_frame = drawDotsAndArrows(prev_gray, flow, blockSize)

    # Convert the frame into RGB (DeepFace uses RGB, OpenCV uses BGR)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform facial detection and emotion recognition every 10 frames
    if frame_count % frame_interval == 0:
        face_data = []

        # Face detection with MTCNN
        faces = detector.detect_faces(rgb_frame)

        for face in faces:
            # Extract face's coordinates
            x, y, w, h = face['box']
            
            # Crop the face from the frame
            face_img = rgb_frame[y:y + h, x:x + w]

            # Emotion detection with DeepFace
            emotion_analysis = DeepFace.analyze(face_img, actions=['emotion'], enforce_detection=False)

            # Extract the dominant emotion, its score and the color associated to it
            dominant_emotion = emotion_analysis[0]['dominant_emotion']
            emotion_score = emotion_analysis[0]['emotion'][dominant_emotion]
            emotion_color = emotion_colors.get(dominant_emotion.lower())

            # Save data of the face
            face_data.append({
                "box": (x, y, w, h),
                "emotion": dominant_emotion,
                "score": emotion_score,
                "color": emotion_color
                })

    #Draw rectangles and text on the frame
    for data in face_data:
        x, y, w, h = data["box"]
        emotion = data["emotion"]
        score = data["score"]
        color = data["color"]

        # Add a background rectangle for the text
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.2
        thickness = 2

        # Compute the size of the text to add a background rectangle
        (text_width, text_height), _ = cv2.getTextSize(f'{dominant_emotion}: {emotion_score:.0f}', font, font_scale, thickness)

        # Create a rectangle for the text
        cv2.rectangle(frame, (x, y - 35), (x + text_width + 30, y), color, -1)

        # Write the text on the rectangle
        cv2.putText(frame, f'{emotion}: {score:.0f}%', (x, y - 5), font, font_scale, (255, 255, 255), thickness)

        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

        # Draw circles on the corners of the rectangle
        radius = 7
        cv2.circle(frame, (x, y), radius, color, -1)  # Top-left corner
        cv2.circle(frame, (x + w, y), radius, color, -1)  # Top-right corner
        cv2.circle(frame, (x, y + h), radius, color, -1)  # Bottom-left corner
        cv2.circle(frame, (x + w, y + h), radius, color, -1)  # Bottom-right corner

    
    # Show the frame in real time 
    cv2.imshow("Emotion Recognition - Live", np.hstack((frame, flow_frame)))

    # Quit the loop if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Increase the frame counter
    frame_count += 1

# Release the webcam and close the windows
cap.release()
cv2.destroyAllWindows()
